# 50.007 Machine Learning Project Part 3

____

### Imports and File Reads

In [2]:
import math
import heapq
from collections import defaultdict
from itertools import groupby

# Read data from files using UTF-8 encoding
with open("Data/ES/train", encoding="utf-8") as f:
    es = f.read().splitlines()
with open("Data/RU/train", encoding="utf-8") as f:
    ru = f.read().splitlines()
with open("Data/ES/dev.in", encoding="utf-8") as f:
    dev_in_es = f.read().splitlines()
with open("Data/ES/dev.out", encoding="utf-8") as f:
    dev_out_es = f.read().splitlines()


### Estimating Emission Parameters for Hidden Markov Model

In [5]:
def estimate_emission_params(train_data, k=1):
    word_sentiment_counts = defaultdict(lambda: defaultdict(int))
    sentiment_counts = defaultdict(int)
    emission_params = {}

    # Count occurrences of (label, word) pairs
    for sentence in train_data:
        try:
            if sentence != "":
                x, label = sentence.split(" ")
        except:
            continue

        sentiment_counts[label] += 1
        word_sentiment_counts[label][x] += 1

    # Calculate emission parameters
    for key in word_sentiment_counts:
        for word in word_sentiment_counts[key]:
            emission_params[(word, key)] = word_sentiment_counts[key][word] / sentiment_counts[key]

    return emission_params, sentiment_counts

# Estimate emission parameters
es_para, count = estimate_emission_params(es)
display(es_para)
print("Emission Parameters for ('palo', 'O'): ", es_para[("palo", "O")])

# Extract states
states = list(count.keys())
print("States:", states)


{('Estuvimos', 'O'): 0.0001943193963144088,
 ('hace', 'O'): 0.000842050717362438,
 ('poco', 'O'): 0.0017812611328820805,
 ('mi', 'O'): 0.0023318327557729055,
 ('pareja', 'O'): 0.000421025358681219,
 ('y', 'O'): 0.0331962302037115,
 ('yo', 'O'): 0.0011659163778864527,
 ('comiendo', 'O'): 0.00032386566052401467,
 ('resultó', 'O'): 0.00012954626420960585,
 ('todo', 'O'): 0.00375684166207857,
 ('muy', 'O'): 0.01282508015675098,
 ('bien', 'O'): 0.005538102794960651,
 (',', 'O'): 0.05398840560935324,
 ('tanto', 'O'): 0.0012630760760436572,
 ('la', 'O'): 0.024451857369563106,
 ('el', 'O'): 0.02079217540564174,
 ('…', 'O'): 0.001684101434724876,
 ('nos', 'O'): 0.0047284386436506134,
 ('gustó', 'O'): 0.0003562522265764161,
 ('mucho', 'O'): 0.0017164880007772775,
 ('.', 'O'): 0.10360462480163228,
 ('Por', 'O'): 0.0009715969815720439,
 ('poner', 'O'): 0.00016193283026200733,
 ('algún', 'O'): 0.0001943193963144088,
 ('pero', 'O'): 0.0061858341160086795,
 ('quizá', 'O'): 0.00012954626420960585,
 ('

Emission Parameters for ('palo', 'O'):  3.238656605240146e-05
States: ['O', 'B-positive', 'B-negative', 'B-neutral', 'I-neutral', 'I-positive', 'I-negative']


### Estimating Transition Probabilities for Hidden Markov Model


In [4]:
def estimate_transition_parameters_test(sentences):
    transition_counts = {}
    state_counts = {}

    # Group sentences
    list_of_sentences = [list(sub) for ele, sub in groupby(sentences, key=bool) if ele]

    # Count state and transition occurrences
    for one_sentence in list_of_sentences:
        prev_state = 'START'

        for one_word in one_sentence:
            if one_word != "":
                word, state = one_word.split(" ")

                # Count state occurrences
                if state_counts.get(prev_state):
                    state_counts[prev_state] += 1
                else:
                    state_counts[prev_state] = 1

                # Count transition occurrences
                if prev_state not in transition_counts:
                    transition_counts[prev_state] = {}
                if state not in transition_counts[prev_state]:
                    transition_counts[prev_state][state] = 1
                else:
                    transition_counts[prev_state][state] += 1

                prev_state = state

        # Count 'END' transitions
        if "END" not in transition_counts[prev_state]:
            transition_counts[prev_state]["END"] = 1
        else:
            transition_counts[prev_state]["END"] += 1

    # Calculate transition probabilities
    for from_state, to_states in transition_counts.items():
        total_from_state_count = state_counts[from_state]
        for to_state, count in to_states.items():
            transition_counts[from_state][to_state] = count / total_from_state_count

    return transition_counts

# Estimate transition parameters
transition_params = estimate_transition_parameters_test(es)
print("Transition Parameters:", transition_params)


Transition Parameters: {'START': {'O': 0.9289176090468497, 'B-positive': 0.052234787291330104, 'B-negative': 0.014001077005923533, 'B-neutral': 0.004846526655896607}, 'O': {'O': 0.9456845511712573, 'B-positive': 0.038980620012503214, 'END': 0.06773802081418012, 'B-negative': 0.013054830287206266, 'B-neutral': 0.002279998529033207}, 'B-positive': {'O': 0.8791304347826087, 'I-positive': 0.11739130434782609, 'END': 0.008695652173913044, 'B-neutral': 0.0008695652173913044, 'B-positive': 0.0026086956521739132}, 'B-negative': {'O': 0.8196286472148541, 'I-negative': 0.18037135278514588, 'END': 0.010610079575596816}, 'B-neutral': {'I-neutral': 0.20833333333333334, 'O': 0.7916666666666666}, 'I-neutral': {'I-neutral': 0.6511627906976745, 'O': 0.3488372093023256}, 'I-positive': {'I-positive': 0.5718849840255591, 'O': 0.4281150159744409, 'END': 0.003194888178913738}, 'I-negative': {'O': 0.39766081871345027, 'I-negative': 0.6023391812865497}}


### Enhanced Viterbi Algorithm with Backpointers for Sequence Tagging

In [5]:
def vertibi_algorithm_test(sentence, transition_params, emission_params, states):
    n = len(sentence)
    num_states = len(states)
    viterbi = [{} for _ in range(n)]
    backpointers = [{} for _ in range(n)]

    # Initialization at time step 0
    word = sentence[0]
    for i in range(num_states):
        viterbi[0][states[i]] = math.log(emission_params.get((word, states[i]), 1e-10) + transition_params["START"].get(states[i], 1e-10))
    
    # Forward pass
    for z in range(1, n):
        word = sentence[z]
        for i in range(num_states):
            max_prob = float('-inf')
            save_state = ""
            for j in range(num_states):
                prob = viterbi[z - 1].get(states[j], float('-inf')) + math.log(emission_params.get((word, states[i]), 1e-10) + transition_params[states[j]].get(states[i], 1e-10))
                if prob > max_prob:
                    max_prob = prob
                    save_state = states[j]
            viterbi[z][states[i]] = max_prob
            backpointers[z][states[i]] = save_state

    # Termination step
    max_prob = float('-inf')
    final_state = 'STOP'
    for j in range(num_states):
        prob = viterbi[n - 1].get(states[j], float('-inf')) + math.log(transition_params[states[j]].get("STOP", 1e-10))
        if prob > max_prob:
            max_prob = prob
            final_state = states[j]
    
    # Backtracking step
    best_path = [final_state]
    for m in range(n - 1, 0, -1):
        best_path.insert(0, backpointers[m][best_path[0]])
    
    return best_path

def run_viterbi_on_dev_set(dev_set, transition_params, emission_params, states):
    output = []
    list_of_sentences = [list(sub) for ele, sub in groupby(dev_set, key=bool) if ele]
    for sentence in list_of_sentences:
        best_path = vertibi_algorithm_test(sentence, transition_params, emission_params, states)
        output.append(best_path)

    return output

### Model Evaluation Metrics

In [6]:
def compute_metrics(true_tags, predicted_tags):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    
    for true_seq, pred_seq in zip(true_tags, predicted_tags):
        for true, pred in zip(true_seq, pred_seq):
            # Print for debugging purposes
            print(true, pred, true == pred)
            
            if true == pred and true != 'O':
                true_positive += 1
                print("True Positive Count:", true_positive)
            elif true != pred and true != 'O' and pred != 'O':
                false_positive += 1
                false_negative += 1  # Counting false negatives
    
    print("True Positive:", true_positive)
    print("False Positive:", false_positive)
    print("False Negative:", false_negative)
    
    # Calculate precision, recall, and F-score
    precision = true_positive / (true_positive + false_positive) if true_positive + false_positive > 0 else 0
    recall = true_positive / (true_positive + false_negative) if true_positive + false_negative > 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    
    return precision, recall, f_score


### Precision and F-score Calculation Functions

In [7]:
# Calculate precision from true and predicted labels
def calculate_precision(true_labels, predicted_labels):
    correct_count = 0
    total_count = 0
    
    for true_seq, pred_seq in zip(true_labels, predicted_labels):
        for true, pred in zip(true_seq, pred_seq):
            if true == pred:
                correct_count += 1
            total_count += 1
    
    precision = correct_count / total_count if total_count > 0 else 0
    return precision

def calculate_recall(true_labels, predicted_labels):
    true_positive = 0
    false_negative = 0
    
    for true_seq, pred_seq in zip(true_labels, predicted_labels):
        for true, pred in zip(true_seq, pred_seq):
            if true == pred and true != 'O':
                true_positive += 1
            elif true != pred and true != 'O':
                false_negative += 1
    
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return recall

# Calculate F-score using recall and precision
def calculate_f_score(recall, precision):
    v1 = 1 / precision if precision > 0 else 1
    v2 = 1 / recall if recall > 0 else 1
    f_score = 2 / (v1 + v2) if (v1 + v2) > 0 else 0
    return f_score


### Enhanced Viterbi Algorithm with Backpointers for Analysis Tagging

In [8]:
# Viterbi Algorithm Implementation with Backpointers
def viterbi_algorithm_2(sentence, transition_params, emission_params, states):
    n = len(sentence)
    num_states = len(states)
    viterbi = [{} for _ in range(n)]
    backpointers = [{} for _ in range(n)]

    # Initialization at time step 0
    for state in states:
        emission_prob = emission_params.get((sentence[0], state), 1e-10)
        viterbi[0][state] = math.log(transition_params['START'].get(state, 1e-10)) + math.log(emission_prob)
        backpointers[0][state] = 'START'

    # Forward pass
    for t in range(1, n):
        for state in states:
            max_prob = float('-inf')
            prev_state = None
            for prev_state in states:
                transition_prob = transition_params[prev_state].get(state, 1e-10)
                emission_prob = emission_params.get((sentence[t], state), 1e-10)
                prob = viterbi[t - 1].get(prev_state, 1e-10) + math.log(transition_prob) + math.log(emission_prob)
                if prob > max_prob:
                    max_prob = prob
                    backpointers[t][state] = prev_state
            viterbi[t][state] = max_prob

    # Termination step
    max_prob = float('-inf')
    final_state = None
    for state in states:
        transition_prob = transition_params[state].get('STOP', 1e-10)
        prob = viterbi[n - 1][state] + math.log(transition_prob)
        if prob > max_prob:
            max_prob = prob
            final_state = state

    # Backtracking step
    best_path = [final_state]
    for t in range(n - 1, 0, -1):
        best_path.insert(0, backpointers[t][best_path[0]])

    return best_path

# Run Viterbi algorithm on the development set using viterbi_algorithm_2
def run_viterbi_on_dev_set_2(dev_set, transition_params, emission_params, states):
    output = []
    list_of_sentences = [list(sub) for ele, sub in groupby(dev_set, key=bool) if ele]
    for sentence in list_of_sentences:
        best_path = viterbi_algorithm_2(sentence, transition_params, emission_params, states)
        output.append(best_path)

    return output


In [9]:
# Train the model on the training set
transition_params = estimate_transition_parameters_test(es)
emission_params, count = estimate_emission_params(es)

# Extract the list of states from the count dictionary
states = list(count.keys())

# Run Viterbi algorithm on the development set using both implementations
predicted_tags = run_viterbi_on_dev_set(dev_in_es, transition_params, emission_params, states)
predicted_tags_2 = run_viterbi_on_dev_set_2(dev_in_es, transition_params, emission_params, states)

# Function to extract actual tags from a test set
def actual_tags(test_set):
    tags = []
    list_of_sentences = [list(sub) for ele, sub in groupby(test_set, key=bool) if ele]
    for sentence in list_of_sentences:
        innerlist = []
        for word in sentence:
            w, state = word.split()
            innerlist.append(state)
        tags.append(innerlist)
    return tags

# Calculate precision using the scores function
precision = calculate_precision(actual_tags(dev_out_es), predicted_tags)
print("Precision using implementation 1:", precision)

precision_2 = calculate_precision(actual_tags(dev_out_es), predicted_tags_2)
print("Precision using implementation 2:", precision_2)

recall = calculate_recall(actual_tags(dev_out_es), predicted_tags)
print("Recall using implementation 1:", recall)

recall_2 = calculate_recall(actual_tags(dev_out_es), predicted_tags_2)
print("Recall using implementation 2:", recall_2)

# Calculate F-score using the scores function
f_score = calculate_f_score(recall, precision)
print("F-score using implementation 1:", f_score)

f_score_2 = calculate_f_score(recall_2, precision_2)
print("F-score using implementation 2:", f_score_2)

Precision using implementation 1: 0.9271799628942486
Precision using implementation 2: 0.935064935064935
Recall using implementation 1: 0.0
Recall using implementation 2: 0.42038216560509556
F-score using implementation 1: 0.9622141997593262
F-score using implementation 2: 0.5800073233247894


In [10]:
def k_best_viterbi(sentence, transition_params, emission_params, states, k):
    n = len(sentence)
    num_states = len(states)
    k_best = [[] for _ in range(n)]
    backpointers = [{} for _ in range(n)]
    
    # Initialization at time step 0
    word = sentence[0]
    for i in range(num_states):
        emission_prob = emission_params.get((word, states[i]), 1e-10)
        k_best[0].append((math.log(emission_prob) + math.log(transition_params["START"].get(states[i], 1e-10)), [states[i]]))
    
    # Forward pass
    for z in range(1, n):
        word = sentence[z]
        for i in range(num_states):
            max_probs = []  # Store top-k probabilities
            save_states = []  # Store corresponding sequences
            
            for j in range(num_states):
                prev_state = states[j]
                emission_prob = emission_params.get((word, states[i]), 1e-10)
                prob = math.log(emission_prob) + math.log(transition_params[prev_state].get(states[i], 1e-10))
                
                for p, seq in k_best[z - 1]:
                    new_seq = seq + [states[i]]
                    heapq.heappush(max_probs, (p + prob, new_seq))
                    if len(max_probs) > k:
                        heapq.heappop(max_probs)
            
            k_best[z] = max_probs
            backpointers[z][states[i]] = [seq for _, seq in max_probs]
    
    # Termination step
    max_probs = []
    final_state = 'STOP'
    for j in range(num_states):
        prob = math.log(transition_params[states[j]].get("STOP", 1e-10))
        for p, seq in k_best[n - 1]:
            new_seq = seq + [final_state]
            heapq.heappush(max_probs, (p + prob, new_seq))
            if len(max_probs) > k:
                heapq.heappop(max_probs)
    
    best_paths = [seq for _, seq in max_probs]
    return best_paths



In [11]:
with open("Data/ES/dev.in", encoding="utf-8") as f:
    dev_in_es = f.read().splitlines()
with open("Data/ES/dev.out", encoding="utf-8") as f:
    dev_out_es = f.read().splitlines()

with open("Data/RU/dev.in", encoding="utf-8") as f:
    dev_in_ru = f.read().splitlines()
with open("Data/RU/dev.out", encoding="utf-8") as f:
    dev_out_ru = f.read().splitlines()

k = 8  # Number of top-k sequences to find
for idx, sentence in enumerate(dev_in_es):
    sentence = sentence.split()
    top_k_sequences = k_best_viterbi(sentence, transition_params, emission_params, states, k)
    
    # Select the 2nd and 8th best sequences
    second_best = top_k_sequences[1]
    eighth_best = top_k_sequences[7]
    
    # Convert sequences to the format of the dev_out_es file
    second_best_output = ' '.join(second_best)
    eighth_best_output = ' '.join(eighth_best)
    
    # Print the 2nd and 8th best outputs
    print(f"2nd Best Output for Sentence {idx+1}: {second_best_output}")
    print(f"8th Best Output for Sentence {idx+1}: {eighth_best_output}")

    # Calculate metrics for the 2nd and 8th best outputs
    precision_2 = calculate_precision([dev_out_es[idx]], [second_best])
    recall_2 = calculate_recall([dev_out_es[idx]], [second_best])
    f_score_2 = calculate_f_score(precision_2, recall_2)
    
    precision_8 = calculate_precision([dev_out_es[idx]], [eighth_best])
    recall_8 = calculate_recall([dev_out_es[idx]], [eighth_best])
    f_score_8 = calculate_f_score(precision_8, recall_8)
    
    print(f"2nd Best Precision: {precision_2:.4f}, Recall: {recall_2:.4f}, F-score: {f_score_2:.4f}")
    print(f"8th Best Precision: {precision_8:.4f}, Recall: {recall_8:.4f}, F-score: {f_score_8:.4f}")
    print()
    
# Run k-best Viterbi on RU development set
for idx, sentence in enumerate(dev_in_ru):
    sentence = sentence.split()
    top_k_sequences = k_best_viterbi(sentence, transition_params, emission_params, states, k)
    
    # Select the 2nd and 8th best sequences
    second_best = top_k_sequences[1]
    eighth_best = top_k_sequences[7]
    
    # Convert sequences to the format of the dev_out_ru file
    second_best_output = ' '.join(second_best)
    eighth_best_output = ' '.join(eighth_best)
    
    # Print the 2nd and 8th best outputs
    print(f"2nd Best Output for Sentence {idx+1}: {second_best_output}")
    print(f"8th Best Output for Sentence {idx+1}: {eighth_best_output}")

    # Calculate metrics for the 2nd and 8th best outputs
    precision_2 = calculate_precision([dev_out_ru[idx]], [second_best])
    recall_2 = calculate_recall([dev_out_ru[idx]], [second_best])
    f_score_2 = calculate_f_score(precision_2, recall_2)
    
    precision_8 = calculate_precision([dev_out_ru[idx]], [eighth_best])
    recall_8 = calculate_recall([dev_out_ru[idx]], [eighth_best])
    f_score_8 = calculate_f_score(precision_8, recall_8)
    
    print(f"2nd Best Precision: {precision_2:.4f}, Recall: {recall_2:.4f}, F-score: {f_score_2:.4f}")
    print(f"8th Best Precision: {precision_8:.4f}, Recall: {recall_8:.4f}, F-score: {f_score_8:.4f}")
    print()

2nd Best Output for Sentence 1: B-negative STOP
8th Best Output for Sentence 1: B-negative STOP
2nd Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000
8th Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000

2nd Best Output for Sentence 2: O STOP
8th Best Output for Sentence 2: O STOP
2nd Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000
8th Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000

2nd Best Output for Sentence 3: O STOP
8th Best Output for Sentence 3: O STOP
2nd Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000
8th Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000

2nd Best Output for Sentence 4: O STOP
8th Best Output for Sentence 4: O STOP
2nd Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000
8th Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000

2nd Best Output for Sentence 5: O STOP
8th Best Output for Sentence 5: O STOP
2nd Best Precision: 0.0000, Recall: 0.0000, F-score: 1.0000
8th Best Precision: 0.0000, Recall: 0.0000, F-sc

IndexError: list index out of range